In [1]:
#1 DONE      #map drug IDs
#2 DONE      #map virus IDs to their names in simi matrices
#3 DONE      #make sure vi_dr_indi has same drug names as Drugbank drug name for consistency with simi matrix
#4 No need   #add empty entity for investigational drugs (no association with any virus but under c;inical trials for covid19)

#5 DONE      #kegg simi not calculated for all 117 drugs due to 1. missing KEGG ID for 3 drugs 2. too low score with all other drugs (happens for 13 drugs)

#TOTAL 117 drugs (3 have no KEGG IDs...13 have too low simi with all other,so no result in browser after running API link, hence no entry in final matrix)
# no result for drugs with too low (<0.0001) SIMCOMP score..so 13 drugs missed
#check with: http://rest.genome.jp/simcomp2/C06818+D08876+D09390+C07624+D02768/C06818+D08876+D09390+C07624+D02768/cutoff=0.0001

#5.a) DONE     #add missing KEGG IDS 
#Doravirine,Docosanol,Interferon alfa,Asunaprevir,Faldaprevir,Laninamivir octanoate...had no kegg ids..added !
#three drugs: Peginterferon alfacon-1,UC-781,Triazavirin (triazavirindoes'nt even have DrugBank ID and name)...still have no kegg id

#5.b) DONE   #handle drugs wiht NAns as similarity (just equate to 0) and add 0 rows/columns for 13 drugs having SIMCOMP<cutoff
#6 DONE   #check the no of drugs in simi matrix (114) and no of drugs in drugs matadata file (117)

#recompute vi sim after replacing segments 
import pandas as pd
import scipy.io as sio
import numpy as np
import math

In [2]:
vi_dr_indi=pd.read_excel('data_raw/database.xlsx', sheet_name='Indications(vi-dr)')

In [3]:
vi_dr_indi=vi_dr_indi[['VIRUS', 'DRUG']]

In [4]:
no_v = len(vi_dr_indi['VIRUS'].unique() )
no_d= len(vi_dr_indi['DRUG'].unique() )

In [5]:
no_v
no_d

121

#### creating interaction matrix

In [6]:
vi_dr_matrix=vi_dr_indi.groupby(['VIRUS', 'DRUG']).size().unstack().fillna(0) #https://www.drawingfromdata.com/making-a-pairwise-distance-matrix-with-pandas

In [7]:
vi_dr_matrix=vi_dr_matrix.sort_index(axis=1)
vi_dr_matrix.columns

Index(['Abacavir', 'Acyclovir', 'Adefovir', 'Adefovir dipivoxil', 'Amantadine',
       'Amprenavir', 'Asunaprevir', 'Atazanavir', 'Baloxavir marboxil',
       'Beclabuvir',
       ...
       'Valaciclovir', 'Valganciclovir', 'Valomaciclovir', 'Vaniprevir',
       'Velpatasvir', 'Vidarabine', 'Voxilaprevir', 'Zalcitabine', 'Zanamivir',
       'Zidovudine'],
      dtype='object', name='DRUG', length=121)

In [8]:
vi_dr_matrix.to_csv('data_processed/virus_drug_association.csv', header=False, index=False)

sio.savemat('data_processed/virus_drug_association.mat',{'mat':vi_dr_matrix.values , 'vi':vi_dr_matrix.index.values, 'dr':vi_dr_matrix.columns.values})

In [9]:
vi_dr_matrix.index.values.T

array(['Chikungunya virus', 'Coxsackievirus B5', 'Ebola virus',
       'Enterovirus D', 'Enterovirus J', 'HBV', 'HCV', 'HHV-1', 'HHV-2',
       'HHV-3', 'HHV-4', 'HHV-5', 'HHV-6', 'HHV-8', 'HIV-1', 'HIV-2',
       'HPIV-2', 'HPIV-3', 'HPV', 'HRV-A', 'HRV-B', 'HTLV-1',
       'Hantaan orthohantavirus', 'Hendra virus', 'Human adenovirus',
       'Human polyomavirus', 'Influenza A virus', 'Influenza B virus',
       'Influenza C virus', 'Lassa mammarenavirus', 'MCV', 'MERS-CoV',
       'Nipah virus', 'RSV', 'SARS-CoV', 'SARS-CoV-2', 'VV',
       'Variola virus', 'Zika virus'], dtype=object)

In [10]:
vi_dr_matrix.columns

Index(['Abacavir', 'Acyclovir', 'Adefovir', 'Adefovir dipivoxil', 'Amantadine',
       'Amprenavir', 'Asunaprevir', 'Atazanavir', 'Baloxavir marboxil',
       'Beclabuvir',
       ...
       'Valaciclovir', 'Valganciclovir', 'Valomaciclovir', 'Vaniprevir',
       'Velpatasvir', 'Vidarabine', 'Voxilaprevir', 'Zalcitabine', 'Zanamivir',
       'Zidovudine'],
      dtype='object', name='DRUG', length=121)

### Creating drug similarity matrix
* 1. get KEGG IDs for the drugs
* 2. add KEGG IDs manually for the drugs which dont have KEGG IDs in mapping file (drug links.csv) obtained from DrugBank
* 3. use rest API to get SIMCOMP score: https://www.genome.jp/tools/gn_tools_api.html   to get SIMCOMP similarities using KEGG IDs
* 4. convert similarity score fetched from API to matrix

##### 1. get KEGG IDs for the drugs

In [11]:
#FETCH KEGG IDS AND WRITE TO NEW UPDATES DRUGS FILE with_KEGG_IDs

#drugs=pd.read_excel('data_raw/database.xlsx', sheet_name='Drug_info')
#mapping_file=pd.read_csv('data_raw/drug links.csv') #obtained from https://www.drugbank.ca/releases/latest#external-links  : has wrong drug linking for oseltamivir..corrected

#drugs_with_KEGG_IDs = pd.merge(mapping_file, drugs, left_on='DrugBank ID',right_on='DRUGBANK ID', how='right')
#drugs_with_KEGG_IDs=drugs_with_KEGG_IDs[['DRUGBANK ID','DrugBank ID','Name','KEGG Compound ID', 'KEGG Drug ID']].sort_values('DRUGBANK ID')
#drugs_with_KEGG_IDs.to_csv('data_processed/drugs_withKEGGIDs.csv', index=False)

##### 2. Add KEGG IDs of drusg with missing IDs manually

##### 3. use rest API to get SIMCOMP score:

In [12]:
#read
drugs_KEGGSupdated=pd.read_excel('data_raw/database.xlsx', sheet_name='Drug_info')#118 drugs (KEGG IDs of 3 drugs missing)
drugs_with_KEGG_IDs=drugs_KEGGSupdated[['DRUGBANK ID','DrugBank ID','Name','KEGG Compound ID', 'KEGG Drug ID']].sort_values(by=['KEGG Compound ID','KEGG Drug ID'])

In [13]:
#all drugs
dr_list1=drugs_with_KEGG_IDs['Name'].values.tolist() #117 drugs

#drugs having KEGG ID
'''drugs_with_KEGG_IDs['KEGG Combined ID']=drugs_with_KEGG_IDs['KEGG Compound ID'].fillna('') + drugs_with_KEGG_IDs['KEGG Drug ID'].fillna('')
drugs_with_KEGG_IDs['KEGG Combined ID'].replace('', np.nan, inplace=True)
dr_list2a=drugs_with_KEGG_IDs.dropna(subset = ['KEGG Combined ID']) #114 drugs'''

"drugs_with_KEGG_IDs['KEGG Combined ID']=drugs_with_KEGG_IDs['KEGG Compound ID'].fillna('') + drugs_with_KEGG_IDs['KEGG Drug ID'].fillna('')\ndrugs_with_KEGG_IDs['KEGG Combined ID'].replace('', np.nan, inplace=True)\ndr_list2a=drugs_with_KEGG_IDs.dropna(subset = ['KEGG Combined ID']) #114 drugs"

In [14]:
cmnd=""
dr_list2=[] # 101 drugs (13 drugs had too low SIMCOMP score with all other 114 drugs)
for i in range(len(drugs_with_KEGG_IDs.index)):
    nonans=drugs_with_KEGG_IDs.iloc[i].isnull().sum()  
    
    if nonans==1:
               
        if pd.isnull(drugs_with_KEGG_IDs.iloc[i, 3]):
            #print(drugs_with_KEGG_IDs.iloc[i, 4])
            cmnd=cmnd+ drugs_with_KEGG_IDs.iloc[i, 4]
            
        else:
            #print(drugs_with_KEGG_IDs.iloc[i, 3])
            cmnd=cmnd+ drugs_with_KEGG_IDs.iloc[i, 3]
        dr_list2.append(drugs_with_KEGG_IDs.iloc[i, 2])
        cmnd=cmnd+ '+'
    
    elif nonans==0:
        
        #print(drugs_with_KEGG_IDs.iloc[i, 3])
        cmnd=cmnd+ drugs_with_KEGG_IDs.iloc[i, 3]
        dr_list2.append(drugs_with_KEGG_IDs.iloc[i, 2])
        cmnd=cmnd+ '+'  
    
print(cmnd) #lists all drug KEGG IDs (114) except those which have both NANs in KEGG drug and compund ID
#len(dr_list2)#same as drug_list2a (drugs having KEGG ID)

C01588+C05682+C06456+C06810+C06818+C06909+C06941+C06953+C07043+C07051+C07065+C07184+C07195+C07207+C07210+C07236+C07240+C07257+C07263+C07312+C07417+C07624+C07625+C08086+C08088+C08092+C08095+C09147+C10874+C11222+C11277+C12599+C12871+C13480+C15643+C17407+C21598+D00317+D00333+D00342+D00391+D00423+D00429+D00745+D01784+D01982+D01995+D02495+D02496+D02497+D02499+D02500+D02736+D02737+D02744+D02747+D02748+D02768+D03305+D03537+D03656+D03829+D03884+D03956+D03981+D04008+D04112+D04552+D04554+D04859+D05528+D06275+D06651+D06670+D06675+D06676+D06677+D07199+D07249+D07250+D07471+D08605+D08876+D08997+D09012+D09390+D09410+D09537+D09575+D09720+D09881+D09971+D09987+D10066+D10081+D10093+D10105+D10366+D10428+D10442+D10462+D10558+D10580+D10582+D10610+D10624+D10625+D10639+D10745+D10801+D10806+D10814+D10816+D10899+D10909+D11021+D11399+D11472+


In [15]:
set(dr_list1)-set(dr_list2)

{'Peginterferon alfacon-1', 'Triazavirin', 'UC-781'}

Copy paste the following command

In [16]:
#http://rest.genome.jp/simcomp2/C01588+C05682+C06456+C06810+C06818+C06909+C06941+C06953+C07043+C07051+C07065+C07184+C07195+C07207+C07210+C07236+C07240+C07257+C07263+C07312+C07417+C07624+C07625+C08086+C08088+C08092+C08095+C09147+C10874+C11222+C11277+C12599+C12871+C13480+C15643+C17407+C21598+D00317+D00333+D00342+D00391+D00423+D00429+D00745+D01784+D01982+D01995+D02495+D02496+D02497+D02499+D02500+D02736+D02737+D02744+D02747+D02748+D02768+D03305+D03537+D03656+D03829+D03884+D03956+D03981+D04008+D04112+D04552+D04554+D04859+D05528+D06275+D06651+D06670+D06675+D06676+D06677+D07199+D07249+D07250+D07471+D08605+D08876+D08997+D09012+D09390+D09410+D09537+D09575+D09720+D09881+D09971+D09987+D10066+D10081+D10093+D10105+D10366+D10428+D10442+D10462+D10558+D10580+D10582+D10610+D10624+D10625+D10639+D10745+D10801+D10806+D10814+D10816+D10899+D10909+D11021+D11399+D11472/C01588+C05682+C06456+C06810+C06818+C06909+C06941+C06953+C07043+C07051+C07065+C07184+C07195+C07207+C07210+C07236+C07240+C07257+C07263+C07312+C07417+C07624+C07625+C08086+C08088+C08092+C08095+C09147+C10874+C11222+C11277+C12599+C12871+C13480+C15643+C17407+C21598+D00317+D00333+D00342+D00391+D00423+D00429+D00745+D01784+D01982+D01995+D02495+D02496+D02497+D02499+D02500+D02736+D02737+D02744+D02747+D02748+D02768+D03305+D03537+D03656+D03829+D03884+D03956+D03981+D04008+D04112+D04552+D04554+D04859+D05528+D06275+D06651+D06670+D06675+D06676+D06677+D07199+D07249+D07250+D07471+D08605+D08876+D08997+D09012+D09390+D09410+D09537+D09575+D09720+D09881+D09971+D09987+D10066+D10081+D10093+D10105+D10366+D10428+D10442+D10462+D10558+D10580+D10582+D10610+D10624+D10625+D10639+D10745+D10801+D10806+D10814+D10816+D10899+D10909+D11021+D11399+D11472/cutoff=0.01

#### 4. convert similarity score fetched from API to matrix (after mapping KEGG drug/compund IDs to drug names and adding missed drugs)

In [17]:
drug_sim_raw=pd.read_excel('data_raw/database.xlsx', sheet_name='drug_sim_raw')[['DRUG1','DRUG2','SIMCOMP']]

In [18]:
len(drug_sim_raw['DRUG1'].unique()) #121 total drugs, no KEGG IDS hence no score calculation for 3 drugs, too low SIMCOMP score for 13 drugs..left with 103 drugs

105

In [19]:
#mapping KEGG IDs to drugbank names(some drugbank name missing )
mapping=drugs_with_KEGG_IDs[['KEGG Compound ID', 'Name']].rename(columns={'KEGG Compound ID':'KEGG Drug ID'}).append(drugs_with_KEGG_IDs[['KEGG Drug ID', 'Name']])
dic_dr=dict(mapping.values.tolist())
drug_sim_raw=drug_sim_raw.replace(dic_dr) 

In [20]:
#add 13 drugs having SIMCOMP<cutoff
dr_list3=drug_sim_raw['DRUG1'].unique().tolist() #103 drugs
dr_list_missing = (list(set(dr_list1)-set(dr_list3))) #13 missed drugs+3 drugs with no KEGG ID

In [21]:
for missed_drug in dr_list_missing:
    drug_sim_raw=drug_sim_raw.append({'DRUG1':missed_drug,'DRUG2':missed_drug,'SIMCOMP':1},ignore_index=True)

In [22]:
#convert to matrix
drug_sim_matrix =drug_sim_raw.pivot_table(columns='DRUG1', index='DRUG2', values='SIMCOMP')#.reset_index()
drug_sim_matrix=drug_sim_matrix.fillna(0) #replace NANs (SIMCOP SCORE less than cuoff; eg: http://rest.genome.jp/simcomp2/C06818+D08876+D09390/C06818+D08876+D09390/cutoff=0.01) with 0

In [23]:
drug_sim_matrix

DRUG1,Abacavir,Acyclovir,Adefovir,Adefovir dipivoxil,Amantadine,Amprenavir,Asunaprevir,Atazanavir,Baloxavir marboxil,Beclabuvir,...,Valaciclovir,Valganciclovir,Valomaciclovir,Vaniprevir,Velpatasvir,Vidarabine,Voxilaprevir,Zalcitabine,Zanamivir,Zidovudine
DRUG2,,,,,,,,,,,,,,,,,,,,,
Abacavir,1.000000,0.501832,0.414474,0.414474,0.156250,0.051724,0.151603,0.053352,0.095238,0.120690,...,0.325967,0.412429,0.287313,0.142061,0.181486,0.525424,0.129235,0.324503,0.195929,0.246459
Acyclovir,0.443662,1.000000,0.551724,0.551724,0.000000,0.018868,0.043478,0.059002,0.127273,0.069277,...,0.694215,0.530249,0.260163,0.027397,0.109741,0.617021,0.062740,0.416309,0.075269,0.302817
Adefovir,0.414474,0.551724,1.000000,1.000000,0.000000,0.052632,0.057143,0.057269,0.103448,0.079882,...,0.423841,0.315789,0.223709,0.026667,0.116505,0.520913,0.059850,0.236749,0.000000,0.171171
Adefovir dipivoxil,0.414474,0.551724,1.000000,1.000000,0.000000,0.052632,0.057143,0.057269,0.103448,0.079882,...,0.423841,0.315789,0.223709,0.026667,0.116505,0.520913,0.059850,0.236749,0.000000,0.171171
Amantadine,0.156250,0.000000,0.000000,0.000000,1.000000,0.061571,0.127695,0.000000,0.017544,0.116667,...,0.000000,0.000000,0.000000,0.092308,0.095541,0.000000,0.091160,0.102662,0.000000,0.122449
Amprenavir,0.051724,0.018868,0.052632,0.052632,0.061571,1.000000,0.262003,0.339286,0.257669,0.287129,...,0.205534,0.181989,0.131034,0.271754,0.251422,0.050725,0.185270,0.142241,0.151631,0.122047
Asunaprevir,0.133429,0.043478,0.057143,0.057143,0.102107,0.248304,1.000000,0.334981,0.234568,0.276347,...,0.094183,0.091644,0.072961,0.529093,0.318229,0.055556,0.504975,0.092308,0.103253,0.121076
Atazanavir,0.053352,0.059002,0.057269,0.057269,0.000000,0.339286,0.365360,0.927273,0.250000,0.264775,...,0.127379,0.109551,0.139535,0.264237,0.407821,0.055635,0.307692,0.059908,0.098266,0.067251
Baloxavir marboxil,0.095238,0.127273,0.103448,0.103448,0.017544,0.253823,0.234568,0.235813,1.000000,0.253165,...,0.131148,0.126984,0.097561,0.194896,0.305310,0.064516,0.211790,0.144465,0.000000,0.132404


In [24]:
drug_sim_matrix.to_csv('data_processed/similarity_drugs.csv', index=True)

sio.savemat('data_processed/drug_sim_matrix.mat',{'Sd':drug_sim_matrix.values , 'dr_names':drug_sim_matrix.index.values})

### Creating virus similarity matrix
* 1. fetch complete genome for viruses (preferably refseq) from NCBI as fasta files (data_raw/virus_completeGenome/)
* 2. find ONF (k-mer) based dissimilarities measures between them (data_processed/ONF_distance_shalini_system/ using VirHostMatcher) #has an empty last colmn
* 3. subtract from 1 to get the similarity
* 4. map viral genomic sequnce id to virus name

##### 1. fetch complete genome for viruses: done manually

##### 2. find ONF based distance measure: ran VirhostMatcher on Linux machine (Shalini's system)

##### 3. subtract from 1 to get the similarity

In [25]:
vir_sim_matrix=1-pd.read_csv('data_processed/ONF_distance/d2star_k6.csv', index_col=0, header=0)

vir_sim_matrix= vir_sim_matrix.iloc[:, :-1] #Remove last empty col
vir_sim_matrix.columns=vir_sim_matrix.columns.str.rstrip('.fasta') #strip .fasta suffexi from col headers
vir_sim_matrix.index=vir_sim_matrix.index.str.rstrip('.fasta') #strip .fasta suffexi from row indices
vir_sim_matrix

FileNotFoundError: File b'data_processed/ONF_distance/d2star_k6.csv' does not exist

##### Map viral genomic sequnce id to virus name

In [ ]:
mapping=pd.read_excel('data_raw/database.xlsx', sheet_name='Virus_info')[['NCBI RefSeq Accession ID','ACRONYM' ]]
dic_vi=dict(mapping.values.tolist())
vir_sim_matrix=vir_sim_matrix.rename(columns=dic_vi, index=dic_vi) 


In [ ]:
vir_sim_matrix=vir_sim_matrix.sort_index(axis=1)
vir_sim_matrix=vir_sim_matrix.sort_index(axis=0)
vir_sim_matrix

In [ ]:
vir_sim_matrix.to_csv('data_processed/similarity_virus.csv')

sio.savemat('data_processed/vir_sim_matrix.mat',{'Sv':vir_sim_matrix.values , 'vi_names':vir_sim_matrix.index.values})

In [ ]:
vi_dr_matrix

In [ ]:
sio.savemat('data_processed/virus_drug_association_withcv.mat',{'mat':vi_dr_matrix.values , 'vi':vi_dr_matrix.index.values, 'dr':vi_dr_matrix.columns.values})
sio.savemat('data_processed/vir_sim_matrix_withcv.mat',{'Sv':vir_sim_matrix.values , 'vi_names':vir_sim_matrix.index.values})
sio.savemat('data_processed/drug_sim_matrix_withcv.mat',{'Sd':drug_sim_matrix.values , 'dr_names':drug_sim_matrix.index.values})

#REMOVE SARS-CoV-2 
vi_dr_matrix=vi_dr_matrix.drop(['SARS-CoV-2'])
vi_dr_matrix=vi_dr_matrix.drop(columns=['Hydroxychloroquine', 'Carmofur'])

vir_sim_matrix=vir_sim_matrix.drop(['SARS-CoV-2'])
vir_sim_matrix=vir_sim_matrix.drop(columns=['SARS-CoV-2'])

drug_sim_matrix=drug_sim_matrix.drop(columns=['Hydroxychloroquine', 'Carmofur'])
drug_sim_matrix=drug_sim_matrix.drop(['Hydroxychloroquine', 'Carmofur'])

sio.savemat('data_processed/virus_drug_association.mat',{'mat':vi_dr_matrix.values , 'vi':vi_dr_matrix.index.values, 'dr':vi_dr_matrix.columns.values})
sio.savemat('data_processed/vir_sim_matrix.mat',{'Sv':vir_sim_matrix.values , 'vi_names':vir_sim_matrix.index.values})
sio.savemat('data_processed/drug_sim_matrix.mat',{'Sd':drug_sim_matrix.values , 'dr_names':drug_sim_matrix.index.values})
